In [1]:
import numpy as np # linear algebra
from sklearn.externals import joblib
import tensorflow as tf

In [2]:
# Load scaler
scaler = joblib.load("../models/scaler.save")

MWG_NWG_configurations = [64, 128]
KWG_configurations = [16, 32]
MDIMC_NDIMC_configurations = [8, 16, 32]
MDIMA_NDIMB_configurations = [8, 16, 32]
KWI_configurations = [2, 8]
VWM_VWN_configurations = [1, 2, 4, 8]
STRM_STRN_configurations = [0, 1]
SA_SB_configurations = [0, 1]

configurations = np.array([]).reshape(0,15)
for MWG_NWG_configuration in MWG_NWG_configurations:
    for KWG_configuration in KWG_configurations:
        for MDIMC_NDIMC_configuration in MDIMC_NDIMC_configurations:
            for MDIMA_NDIMB_configuration in MDIMA_NDIMB_configurations:
                for KWI_configuration in KWI_configurations:
                    for VWM_VWN_configuration in VWM_VWN_configurations:
                        for STRM_STRN_configuration in STRM_STRN_configurations:
                            for SA_SB_configuration in SA_SB_configurations:
                                configurations = np.vstack([configurations, [MWG_NWG_configuration, MWG_NWG_configuration, 
                                                                            KWG_configuration, MDIMC_NDIMC_configuration, 
                                                                            MDIMC_NDIMC_configuration, 
                                                                            MDIMA_NDIMB_configuration, 
                                                                            MDIMA_NDIMB_configuration, KWI_configuration, 
                                                                            VWM_VWN_configuration, VWM_VWN_configuration, 
                                                                            STRM_STRN_configuration, 
                                                                            STRM_STRN_configuration,
                                                                            SA_SB_configuration,
                                                                            SA_SB_configuration,0]])
normalized_configurations = scaler.transform(configurations)[:,:-1]

tf.reset_default_graph()

# Number of atributes in training data
n_att = normalized_configurations.shape[1]

# Neurons
n_neurons_1 = 256
n_neurons_2 = 128
n_neurons_3 = 64
n_neurons_4 = 32

# Placeholder
X = tf.placeholder(dtype=tf.float32, shape=[None, n_att])
Y = tf.placeholder(dtype=tf.float32, shape=[None])

# Initializers
sigma = 1
weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

# Hidden weights
W_hidden_1 = tf.Variable(weight_initializer([n_att, n_neurons_1]))
bias_hidden_1 = tf.Variable(bias_initializer([n_neurons_1]))
W_hidden_2 = tf.Variable(weight_initializer([n_neurons_1, n_neurons_2]))
bias_hidden_2 = tf.Variable(bias_initializer([n_neurons_2]))
W_hidden_3 = tf.Variable(weight_initializer([n_neurons_2, n_neurons_3]))
bias_hidden_3 = tf.Variable(bias_initializer([n_neurons_3]))
W_hidden_4 = tf.Variable(weight_initializer([n_neurons_3, n_neurons_4]))
bias_hidden_4 = tf.Variable(bias_initializer([n_neurons_4]))

# Output weights
W_out = tf.Variable(weight_initializer([n_neurons_4, 1]))
bias_out = tf.Variable(bias_initializer([1]))

# Hidden layer
hidden_1 = tf.nn.relu(tf.add(tf.matmul(X, W_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, W_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(tf.add(tf.matmul(hidden_2, W_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(tf.add(tf.matmul(hidden_3, W_hidden_4), bias_hidden_4))

# Output layer (transpose!)
out = tf.transpose(tf.add(tf.matmul(hidden_4, W_out), bias_out))

# Saver
saver = tf.train.Saver()

In [5]:
with tf.Session() as sess:
    
    # Restore variables from disk.
    saver.restore(sess, "../models/model.ckpt")
    print("Model restored.")

    pred = sess.run(out, feed_dict={X: normalized_configurations})
    # print min prediction
    print("Best configuration:")
    print(configurations[pred.argmin()][:-1])

INFO:tensorflow:Restoring parameters from ../models/model.ckpt
Model restored.
Best configuration:
[128. 128.  32.  32.  32.  32.  32.   2.   4.   4.   1.   1.   1.   1.]
